In [41]:
import re
import random
import time

print('Library versions:')

import keras
print(f'keras:{keras.__version__}')
import pandas as pd
print(f'pandas:{pd.__version__}')
import sklearn
print(f'sklearn:{sklearn.__version__}')
import nltk
print(f'nltk:{nltk.__version__}')
import numpy as np
print(f'numpy:{np.__version__}')

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize

from tqdm import tqdm_notebook as tqdm # Special jupyter notebook progress bar 💫

Library versions:
keras:2.8.0
pandas:1.3.5
sklearn:1.0.2
nltk:3.2.5
numpy:1.21.6


In [42]:
import tweepy as tw
import pandas as pd
import numpy as np
import os

# text analysis tools
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import preprocessing
from nltk import SnowballStemmer
import string

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
tqdm().pandas()  # Enable tracking of progress in dataframe `apply` calls

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


0it [00:00, ?it/s]

In [45]:
# 8192 - large enough for demonstration, larger values make network training slower
MAX_VOCAB_SIZE = 2**13
# seq2seq generally relies on fixed length message vectors - longer messages provide more info
# but result in slower training and larger networks
MAX_MESSAGE_LEN = 30  
# Embedding size for words - gives a trade off between expressivity of words and network size
EMBEDDING_SIZE = 100
# Embedding size for whole messages, same trade off as word embeddings
CONTEXT_SIZE = 100
# Larger batch sizes generally reach the average response faster, but small batch sizes are
# required for the model to learn nuanced responses.  Also, GPU memory limits max batch size.
BATCH_SIZE = 4
# Helps regularize network and prevent overfitting.
DROPOUT = 0.2
# High learning rate helps model reach average response faster, but can make it hard to 
# converge on nuanced responses
LEARNING_RATE=0.005

# Tokens needed for seq2seq
UNK = 0  # words that aren't found in the vocab
PAD = 1  # after message has finished, this fills all remaining vector positions
START = 2  # provided to the model at position 0 for every response predicted

# Implementaiton detail for allowing this to be run in Kaggle's notebook hardware
SUB_BATCH_SIZE = 1000

In [46]:
train = pd.read_csv('/content/drive/MyDrive/ADS Group Project/train.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [47]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = text.replace("i'm", "i am")
    text = text.replace("he's", "he is")
    text = text.replace("she's", "she is")
    text = text.replace("it's", "it is")
    text = text.replace("that's", "that is")
    text = text.replace("there's", "there is")
    text = text.replace("what's", "that is")
    text = text.replace(r"where's", "where is")
    text = text.replace("how's", "how is")
    text = text.replace("\'ll", " will")
    text = text.replace("\'ve", " have")
    text = text.replace("\'re", " are")
    text = text.replace("\'d", " would")
    text = text.replace("\'re", " are")
    text = text.replace("won't", "will not")
    text = text.replace("can't", "cannot")
    text = text.replace("n't", " not")
    text = text.replace("n'", "ng")
    text = text.replace("'bout", "about")
    text = text.replace("'til", "until")
    text = text.replace("[-()\"#/@;:<>{}`+=~|.!?,]", "")
    
    return text

In [48]:
tweet_list = train['text'].values.tolist()
translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))


In [49]:
tl=[]
for i in range(len(tweet_list)):
  tl.append(clean_text(tweet_list[i]))

tl1=[]
for i in range(len(tl)):
  w=tl[i].split()
  words=[]
  for n in w:
    if 'http' not in n:
      words.append(n)
  x=' '.join(words)
  tl1.append(x)

In [50]:
stemmer = SnowballStemmer("english")

In [51]:
def tokenize(text):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))  # translator that replaces punctuation with empty spaces
    return [stemmer.stem(i) for i in text.translate(translator).split()]  # stemmer and tokenizing into words

In [52]:
from nltk.corpus import stopwords
nltk.download('stopwords')





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [53]:
stop_words = set(stopwords.words('english'))
#stop_words

In [54]:
stop = set(list(stop_words) + ['\x89u','\x89uo'])
full_stopwords = [tokenize(s)[0] for s in stop]
#full_stopwords

In [55]:
#train.target=train.target.to_string
#train.dtypes

In [56]:
count_vec = CountVectorizer(tokenizer=tokenize, max_features=MAX_VOCAB_SIZE - 3)
print("Fitting CountVectorizer on X and Y text data...")
count_vec.fit(tl1)
analyzer = count_vec.build_analyzer()
vocab = {k: v + 3 for k, v in count_vec.vocabulary_.items()}
vocab['__unk__'] = UNK
vocab['__pad__'] = PAD
vocab['__start__'] = START
# Used to turn seq2seq predictions into human readable strings
reverse_vocab = {v: k for k, v in vocab.items()}
print(f"Learned vocab of {len(vocab)} items.")

Fitting CountVectorizer on X and Y text data...


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


Learned vocab of 8192 items.


In [57]:
def to_word_idx(sentence):
    full_length = [vocab.get(tok, UNK) for tok in analyzer(sentence)] + [PAD] * MAX_MESSAGE_LEN
    return full_length[:MAX_MESSAGE_LEN]

def from_word_idx(word_idxs):
    return ' '.join(reverse_vocab[idx] for idx in word_idxs if idx != PAD).strip()

In [58]:
from_word_idx(to_word_idx(tl1[0]))

'our deed are the reason of this earthquak may allah forgiv us all'

In [59]:
w_idx={i : [to_word_idx(tl1[i]) for i in range(len(tl1))]}

In [60]:
ctrain=pd.DataFrame(data={'tweet':tl1,'target':train.target})
#ctrain['target']=ctrain['target'].to_string



In [61]:
desc=[]
for i in range(len(ctrain)):
  if ctrain.iloc[i]['target']==1:
    desc.append('emergency')
  else:
    desc.append('normal')
ctrain['target_name']=desc

In [62]:
ctrain.loc[ctrain['target']==1]

,tweet,target,target_name
0,our deeds are the reason of this #earthquake m...,1,emergency
1,forest fire near la ronge sask. canada,1,emergency
2,all residents asked to 'shelter in place' are ...,1,emergency
3,"13,000 people receive #wildfires evacuation or...",1,emergency
4,just got sent this photo from ruby #alaska as ...,1,emergency
...,...,...,...
7608,two giant cranes holding a bridge collapse int...,1,emergency
7609,@aria_ahrary @thetawniest the out of control w...,1,emergency
7610,m1.94 [01:04 utc]?5km s of volcano hawaii.,1,emergency
7611,police investigating after an e-bike collided ...,1,emergency


In [63]:
print("Calculating word indexes for X...")
x = pd.np.vstack(ctrain.tweet.progress_apply(to_word_idx).values)
print("Calculating word indexes for Y...")
y = pd.np.vstack(ctrain.target_name.progress_apply(to_word_idx).values)

Calculating word indexes for X...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


  0%|          | 0/7613 [00:00<?, ?it/s]

Calculating word indexes for Y...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.


  0%|          | 0/7613 [00:00<?, ?it/s]

In [64]:
all_idx = list(range(len(x)))
train_idx = set(random.sample(all_idx, int(0.8 * len(all_idx))))
test_idx = {idx for idx in all_idx if idx not in train_idx}

train_x = x[list(train_idx)]
test_x = x[list(test_idx)]
train_y = y[list(train_idx)]
test_y = y[list(test_idx)]

assert train_x.shape == train_y.shape
assert test_x.shape == test_y.shape

print(f'Training data of shape {train_x.shape} and test data of shape {test_x.shape}.')

Training data of shape (6090, 30) and test data of shape (1523, 30).


In [65]:
# keras imports, because there are like... A million of them.
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Input, LSTM, Dropout, Embedding, RepeatVector, concatenate, \
    TimeDistributed
from keras.utils import np_utils

In [66]:
def create_model():
    shared_embedding = Embedding(
        output_dim=EMBEDDING_SIZE,
        input_dim=MAX_VOCAB_SIZE,
        input_length=MAX_MESSAGE_LEN,
        name='embedding',
    )
    
    # ENCODER
    
    encoder_input = Input(
        shape=(MAX_MESSAGE_LEN,),
        dtype='int32',
        name='encoder_input',
    )
    
    embedded_input = shared_embedding(encoder_input)
    
    # No return_sequences - since the encoder here only produces a single value for the
    # input sequence provided.
    encoder_rnn = LSTM(
        CONTEXT_SIZE,
        name='encoder',
        dropout=DROPOUT
    )
    
    context = RepeatVector(MAX_MESSAGE_LEN)(encoder_rnn(embedded_input))
    
    # DECODER
    
    last_word_input = Input(
        shape=(MAX_MESSAGE_LEN, ),
        dtype='int32',
        name='last_word_input',
    )
    
    embedded_last_word = shared_embedding(last_word_input)
    # Combines the context produced by the encoder and the last word uttered as inputs
    # to the decoder.
    decoder_input = concatenate([embedded_last_word, context], axis=2)
    
    # return_sequences causes LSTM to produce one output per timestep instead of one at the
    # end of the intput, which is important for sequence producing models.
    decoder_rnn = LSTM(
        CONTEXT_SIZE,
        name='decoder',
        return_sequences=True,
        dropout=DROPOUT
    )
    
    decoder_output = decoder_rnn(decoder_input)
    
    # TimeDistributed allows the dense layer to be applied to each decoder output per timestep
    next_word_dense = TimeDistributed(
        Dense(int(MAX_VOCAB_SIZE / 2), activation='relu'),
        name='next_word_dense',
    )(decoder_output)
    
    next_word = TimeDistributed(
        Dense(MAX_VOCAB_SIZE, activation='softmax'),
        name='next_word_softmax'
    )(next_word_dense)
    
    return Model(inputs=[encoder_input, last_word_input], outputs=[next_word])

s2s_model = create_model()
optimizer = Adam(lr=LEARNING_RATE, clipvalue=5.0)
s2s_model.compile(optimizer='adam', loss='categorical_crossentropy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [67]:
def add_start_token(y_array):
    """ Adds the start token to vectors.  Used for training data. """
    return np.hstack([
        START * np.ones((len(y_array), 1)),
        y_array[:, :-1],
    ])

def binarize_labels(labels):
    """ Helper function that turns integer word indexes into sparse binary matrices for 
        the expected model output.
    """
    return np.array([np_utils.to_categorical(row, num_classes=MAX_VOCAB_SIZE)
                     for row in labels])

In [68]:
def respond_to(model, text):
    """ Helper function that takes a text input and provides a text output. """
    input_y = add_start_token(PAD * np.ones((1, MAX_MESSAGE_LEN)))
    idxs = np.array(to_word_idx(text)).reshape((1, MAX_MESSAGE_LEN))
    for position in range(MAX_MESSAGE_LEN - 1):
        prediction = model.predict([idxs, input_y]).argmax(axis=2)[0]
        input_y[:,position + 1] = prediction[position]
    return from_word_idx(model.predict([idxs, input_y]).argmax(axis=2)[0])

In [69]:
def train_mini_epoch(model, start_idx, end_idx):
    """ Batching seems necessary in Kaggle Jupyter Notebook environments, since
        `model.fit` seems to freeze on larger batches (somewhere 1k-10k).
    """
    b_train_y = binarize_labels(train_y[start_idx:end_idx])
    input_train_y = add_start_token(train_y[start_idx:end_idx])
    
    model.fit(
        [train_x[start_idx:end_idx], input_train_y], 
        b_train_y,
        epochs=1,
        batch_size=BATCH_SIZE,
    )
    
    rand_idx = random.sample(list(range(len(test_x))), SUB_BATCH_SIZE)
    print('Test results:', model.evaluate(
        [test_x[rand_idx], add_start_token(test_y[rand_idx])],
        binarize_labels(test_y[rand_idx])
    ))
    
    input_strings = [
        "@AppleSupport I fix I this I stupid I problem I",
        "forest fire near la ronge sask. canada",
    ]
    
    for input_string in input_strings:
        output_string = respond_to(model, input_string)
        print(f'> "{input_string}"\n< "{output_string}"')

In [ ]:
training_time_limit = 360 * 60   # seconds (notebooks terminate after 1 hour)
start_time = time.time()
stop_after = start_time + training_time_limit

class TimesUpInterrupt(Exception):
    pass

try:
    for epoch in range(100):
        print(f'Training in epoch {epoch}...')
        for start_idx in range(0, len(train_x), SUB_BATCH_SIZE):
            train_mini_epoch(s2s_model, start_idx, start_idx + SUB_BATCH_SIZE)
            if time.time() > stop_after:
                raise TimesUpInterrupt
except KeyboardInterrupt:
    print("Halting training from keyboard interrupt.")
except TimesUpInterrupt:
    print(f"Halting after {time.time() - start_time} seconds spent training.")

In [ ]:
result=[]
experiment=ctrain.iloc[:500]
for i in range(len(experiment)):
    result.append(respond_to(s2s_model, ctrain.iloc[i]['tweet']))
experiment['result']=result

In [ ]:
experiment

In [ ]:
import sklearn.metrics
#from sklearn.metrics import confusion

sklearn.metrics.confusion_matrix(experiment.target_name,experiment.result)

In [ ]:
cm=experiment.groupby(by=['target_name','result']).agg({'tweet':'count'})

In [ ]:
#print(f'Accuracy is {((cm.iloc[0]+cm.iloc[3])/cm.sum())*100} percent')

In [ ]:
train_xfull = ctrain.tweet[list(train_idx)]
test_xfull = ctrain.tweet[list(test_idx)]
train_yfull = ctrain.target_name[list(train_idx)]
test_yfull = ctrain.target_name[list(test_idx)]

train_2=pd.DataFrame(data={'x':train_xfull,'y':train_yfull})
test_2=pd.DataFrame(data={'x':test_xfull,'y':test_yfull})

In [ ]:
len(test_xfull)

In [ ]:
train_result=[]
for i in range(500):
    train_result.append(respond_to(s2s_model, train_2.iloc[i]['x']))

In [ ]:
test_result=[]
for i in range(500):
    test_result.append(respond_to(s2s_model, test_2.iloc[i]['x']))

In [ ]:
#cm=experiment.groupby(by=['target_name','result']).agg({'tweet':'count'})
#print(f'Accuracy is {((cm.iloc[0]+cm.iloc[3])/cm.sum())*100} percent')

In [ ]:
sklearn.metrics.confusion_matrix(train_yfull[:500],train_result)

In [ ]:
# 97.6 percent in sample

In [ ]:
sklearn.metrics.confusion_matrix(test_yfull[:500],test_result)

In [ ]:
# 75 percent out of sample

In [1]:
TP=int(round(144*1523/500))
TN=int(round(231*1523/500))
FP=int(round(80*1523/500))
FN=int(round(45*1523/500))

In [2]:
print(TP,TN,FP,FN)

439 704 244 137


In [ ]:
import pandas as pd
pred=[]
actual=[]
for i in range(TP):
  pred.append(1)
  actual.append(1)
for i in range(TN):
  pred.append(0)
  actual.append(0)
for i in range(FP):
  pred.append(1)
  actual.append(0)
for i in range(FN):
  pred.append(0)
  actual.append(1)

proj_conf=pd.DataFrame(data={'predicted':pred,'actual':actual})

In [ ]:
class fake(BaseEstimator, RegressorMixin):
    def fit(X, y):
        return y
    
    def predict(X):
        return list(X)


In [ ]:
#fake.predict(proj_conf.predicted)

In [ ]:
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

print(classification_report(actual, pred))
plot_confusion_matrix(fake, proj_conf.predicted, proj_conf.actual, cmap='YlGnBu')